<a href="https://colab.research.google.com/github/iamatul1214/Tensorflow_Certification_Preparation/blob/main/05_TransferLearning_using_tensorflow_part2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Here we will perform transfer learning with fine tuning of our models

In [1]:
import os
Root = "/content/drive/MyDrive/Tensorflow certifications work"
!pwd
os.chdir(Root)
!pwd

/content
/content/drive/MyDrive/Tensorflow certifications work


## We have created a helper function which includes all those small functions we worked in previous modules.

### We can import the functions form the helper.py

In [2]:
try:
  if os.path.exists(os.path.join(os.getcwd(),"Helper.py")):
    print("helper.py exists already, hence skipping downloading..")
  else:
    !wget https://raw.githubusercontent.com/iamatul1214/Tensorflow_Certification_Preparation/main/Helper.py
    print("Downloaded helper.py successfully")
except Exception as e:
  print(e)
  

helper.py exists already, hence skipping downloading..


In [3]:
from Helper import create_tensorboard_callback

## Let's get some data

### This time we will see how we can use the pretrained models frmo `tf.keras.applications` and apply them to our own problems.

In [4]:
## Getting 10% data of of 10 classes of food
from Helper import unzip_data
try:
  if os.path.exists(os.path.join(os.getcwd(),"10_food_classes_10_percent.zip")):
    print(f"data set is already unzipped and present")
  else:
    print("Downloading the data and unzipping it...")
    !wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
    ### unzipping data
    print("Unzipping the data")
    unzip_data("10_food_classes_10_percent.zip")


    
except Exception as e:
  print("data not found in the directory... Hence downloading.....")
  

data set is already unzipped and present


In [5]:
## creating train and test directories
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

In [6]:
## Let's walkthrough the data
from Helper import walk_through_dir
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test

In [7]:
## This time we will not use the ImageDataGenerator, we will use even a smarter version of that.
import tensorflow as tf
IMG_SIZE = (224,224)
BATCH_SIZE = 32
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory = train_dir,
                                                                            image_size = IMG_SIZE,
                                                                            label_mode = "categorical",
                                                                            batch_size = BATCH_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory = test_dir,
                                                                image_size = IMG_SIZE,
                                                                label_mode = "categorical",
                                                                batch_size = BATCH_SIZE)


Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [8]:
train_data_10_percent

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [9]:
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [10]:
## see an example of batches of our data
for images, labels in train_data_10_percent.take(1):  ## 32 is the batch size by default hence the take will take 1 batch
  print(f"Images = {images} and labels = {labels}")
  print(len(labels))

Images = [[[[5.42295914e+01 6.22959185e+00 2.22959185e+00]
   [5.80714302e+01 9.07142925e+00 5.07142973e+00]
   [6.14234695e+01 1.04234686e+01 7.42346811e+00]
   ...
   [4.80919380e+01 8.09193611e+00 6.09193611e+00]
   [5.61325989e+01 1.51325989e+01 1.31325989e+01]
   [4.86122360e+01 7.61223412e+00 5.61223412e+00]]

  [[5.87857132e+01 9.78571415e+00 5.78571415e+00]
   [6.24387779e+01 1.34387779e+01 9.43877792e+00]
   [6.16836739e+01 1.06836739e+01 7.68367386e+00]
   ...
   [4.71377220e+01 6.35198450e+00 4.36728859e+00]
   [4.69847527e+01 5.98475456e+00 3.98475432e+00]
   [5.91838760e+01 1.81838741e+01 1.61838741e+01]]

  [[5.80357132e+01 8.60714340e+00 3.82142901e+00]
   [6.15765305e+01 1.05765305e+01 6.57653046e+00]
   [5.51479607e+01 3.38265324e+00 3.36734861e-01]
   ...
   [4.70253830e+01 6.36208439e+00 2.10190678e+00]
   [5.01122513e+01 9.11225319e+00 5.11225319e+00]
   [4.98928757e+01 8.89287567e+00 4.89287615e+00]]

  ...

  [[2.49357147e+02 2.41142883e+02 2.27714355e+02]
   [2.4

## Model 0: Using transfer learning to build a keras functional API
### The Sequential API runs our layers in the model in sequential manner but functional API is more flexible and can take multiple inputs and outputs.

In [ ]:
## Create base model with tf.keras.applications 
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

## Freeze the base model so that the underlying pretrained models are not retrained again
base_model.trainable = False

## create inputs to our model
inputs = tf.keras.layers.Input(shape = (224,224,3), name = "Input layer")

## If we are using a model like Resnet50V2 we will need to normalize inputs, but not for effiecientNet.
# x = tf.keras.experimental.preprocessing.Rescaling(1./255)(inputs).  ## this means passing the inputs as parameter

x = base_model(inputs)

print(f"The shape of the base model after passing the inputs is {x.shape}")

## Average pool the output of the base_model input layer

x = tf.keras.layers.GlobalAveragePooling2D(name = "global_average_pool_2d") (x)
print(f"The shape of the model after globalaveragepool is {x.shape}")

## create the output activation layer
outputs = tf.keras.layers.Dense(10, activation="softmax", name="output_layer")(x)

## combine the input and output layers
model_0 = tf.keras.Model(inputs,outputs)

## Let's check the summary of the model
print(f"moDEL SUMMARY = {model_0.summary()}")

## compiling the model
model_0.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

## Let's check the summary of the model
print(f"moDEL SUMMARY = {model_0.summary()}")

## fitting the model
from Helper import create_tensorboard_callback
history_0 = model_0.fit(train_data_10_percent,epochs = 3,steps_per_epoch=len(train_data_10_percent),validation_data=test_data,
                        validation_steps = int(0.25 * len(test_data)),callbacks = [create_tensorboard_callback(dir_name = "Transfer_learning",
                                                                                                               experiment_name = "feature_extraction-10_percent")])


## validation_steps = int(0.25 * len(test_data)) will reduce the validation steps to 1/4 and hence reduce training time

16719872/16705208 [==============================] - 0s 0us/step
The shape of the base model after passing the inputs is (None, 7, 7, 1280)
The shape of the model after globalaveragepool is (None, 1280)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pool_2d (Glo  (None, 1280)             0         
 balAveragePooling2D)                                            
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 4,062,381
Trainable params: 12,810
Non-tra

In [ ]:
## Now let's evaluate on full test dataset
model_0.evaluate(test_data)

In [ ]:
model_0.summary()

In [ ]:
base_model.summary()

In [ ]:
## lET'S CHECK OUR TRAINING CURVES
from Helper import plot_loss_curves
plot_loss_curves(history_0)

## Getting the feature vector and understanding the function of GlobalAvgPooling layer.
We had a tensor from base model of (None, 7,7,1280) shape and after the globalAvgPooling layer we got (None, 1280). Let's use a similar shape custom tensor and check what actually GlobalAvgPooling layer does.

In [7]:
import tensorflow as tf
tf.random.set_seed(42)
input_shape = (1,4,4,3)

input_tensor = tf.random.normal(input_shape)
print(f"The random input tensor = \n{input_tensor}\n")
print(f"The shape of the input tensor = \n{input_tensor.shape}")

## let's convert the input_tensor into the globalAvgPool

global_avg_pool_output = tf.keras.layers.GlobalAveragePooling2D()(input_tensor)
print(f"The globalAvgPool output = {global_avg_pool_output}\n")

The random input tensor = 
[[[[ 0.3274685  -0.8426258   0.3194337 ]
   [-1.4075519  -2.3880599  -1.0392479 ]
   [-0.5573232   0.539707    1.6994323 ]
   [ 0.28893656 -1.5066116  -0.26454744]]

  [[-0.59722406 -1.9171132  -0.62044144]
   [ 0.8504023  -0.40604794 -3.0258412 ]
   [ 0.9058464   0.29855987 -0.22561555]
   [-0.7616443  -1.891714   -0.9384712 ]]

  [[ 0.77852213 -0.47338897  0.97772694]
   [ 0.24694404  0.20573747 -0.5256233 ]
   [ 0.32410017  0.02545409 -0.10638497]
   [-0.6369475   1.1603122   0.2507359 ]]

  [[-0.41728497  0.40125778 -1.4145442 ]
   [-0.59318566 -1.6617213   0.33567193]
   [ 0.10815629  0.2347968  -0.56668764]
   [-0.35819843  0.88698626  0.5274477 ]]]]

The shape of the input tensor = 
(1, 4, 4, 3)
The globalAvgPool output = [[-0.09368646 -0.45840445 -0.28855976]]



In [4]:
## So as far as we understood it is taking the avaergage of the features present in input_tensors, Let's see if same output is returned by reduce_mean
tf.reduce_mean(input_tensor=input_tensor, axis = [1,2])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.09368646, -0.45840445, -0.28855976]], dtype=float32)>

In [11]:
## We can try the same thing using the MaxPool2d as well.
max_pool_output  = tf.keras.layers.MaxPool2D()(input_tensor)
print(f"The max pool output = {max_pool_output}")
print(f"The shape of max pool output = {max_pool_output.shape}")

The max pool output = [[[[ 0.8504023  -0.40604794  0.3194337 ]
   [ 0.9058464   0.539707    1.6994323 ]]

  [[ 0.77852213  0.40125778  0.97772694]
   [ 0.32410017  1.1603122   0.5274477 ]]]]
The shape of max pool output = (1, 2, 2, 3)
